In [1]:
from selenium import webdriver

#opt.add_argument('headless')
#opt = webdriver.ChromeOptions()
#dr=webdriver.Chrome('C:/Users/wjssm/Documents/chromedriver.exe',chrome_options = opt) #nothings happen
dr = webdriver.Chrome('C:/Users/wjssm/Documents/chromedriver.exe')


In [164]:
pageNo = str(100)
dr.get('https://music.bugs.co.kr/newest/album/kpoppicked?page='+pageNo)

- 한 페이지당 7 x 10 70개의 앨범
- 대략 100~110 만 긁어보자

`고민거리`
- 새로운 노래가 업데이트 되면 페이지도 달라지는 거 아닌가..?

In [3]:
import pandas as pd
import numpy as np

# 연습

In [165]:
container = dr.find_element_by_id('container')
section = container.find_element_by_css_selector('section[class = "sectionPadding"]')
div  = section.find_element_by_css_selector('div')
lis = div.find_elements_by_css_selector('li')

In [166]:
music_info = {}

## artist

In [167]:
fig = lis[1].find_element_by_css_selector('figcaption')

p = fig.find_element_by_css_selector('p[class = "artist"]')

try : #링크로 걸려있는 artist
    music_info['artist'] = [p.find_element_by_css_selector('a').text]
    
except :
    music_info['artist'] = [p.find_element_by_css_selector('span').text]
music_info


{'artist': ['박지윤']}

## 앨범명

In [168]:
#album title
a = fig.find_element_by_css_selector('a')
music_info['album_title'] = [a.get_attribute('title')]
music_info

{'artist': ['박지윤'], 'album_title': ['Journey']}

## 앨범 클릭

In [169]:
#click 앨범 정보 페이지로 이동
a.click()

## 앨범 정보

In [170]:
div = dr.find_element_by_class_name('basicInfo')
tbody = div.find_element_by_css_selector('tbody')
trs = tbody.find_elements_by_css_selector('tr')

In [171]:
for tr in trs[1:] : #except artist
    th = tr.find_element_by_css_selector('th')
    td = tr.find_element_by_css_selector('td')
    
    music_info[th.text] = [td.text]
    
music_info

{'artist': ['박지윤'],
 'album_title': ['Journey'],
 '앨범 종류': ['싱글'],
 '발매일': ['2018.07.29'],
 '스타일': ['연주 음악'],
 '기획사': ['박지윤크리에이티브'],
 '유통사': ['Sony Music'],
 '재생 시간': ['08:00']}

## 수록곡

In [172]:
section = dr.find_element_by_css_selector('section[class = "sectionPadding contents track"]')
music_info['n'] = [section.find_element_by_css_selector('p').text] #수록곡 개수
music_info

{'artist': ['박지윤'],
 'album_title': ['Journey'],
 '앨범 종류': ['싱글'],
 '발매일': ['2018.07.29'],
 '스타일': ['연주 음악'],
 '기획사': ['박지윤크리에이티브'],
 '유통사': ['Sony Music'],
 '재생 시간': ['08:00'],
 'n': ['(2)']}

In [173]:
table = section.find_element_by_css_selector('table')
tbd = table.find_element_by_css_selector('tbody')
trs = tbd.find_elements_by_css_selector('tr')

In [174]:
music_list = []
for tr in trs :
    ps = tr.find_elements_by_css_selector('p')
    
    aa = []
    #서비스를 지원하지 않는 곡들 처리
    try :
        tr.find_element_by_css_selector('input[disabled = "disabled"]')
    except :
        
        for p in ps :
            
            aa.append(p.text)
        
        music_list.append(aa)

In [175]:
d1 = pd.DataFrame(music_list,columns = ['title_song_or_not','title','artist','bugs_score'])
d1

,title_song_or_not,title,artist,bugs_score
0,1,Letter from,박지윤,선호도 5점
1,1\n[타이틀곡],Journey,박지윤,선호도 5점


In [176]:
music_info.keys()

dict_keys(['artist', 'album_title', '앨범 종류', '발매일', '스타일', '기획사', '유통사', '재생 시간', 'n'])

In [177]:
music_info

{'artist': ['박지윤'],
 'album_title': ['Journey'],
 '앨범 종류': ['싱글'],
 '발매일': ['2018.07.29'],
 '스타일': ['연주 음악'],
 '기획사': ['박지윤크리에이티브'],
 '유통사': ['Sony Music'],
 '재생 시간': ['08:00'],
 'n': ['(2)']}

In [178]:
d2 = pd.DataFrame(music_info)
d2 = pd.DataFrame(np.repeat(np.array(d2),len(d1), axis = 0),
                  columns = music_info.keys())
d2

,artist,album_title,앨범 종류,발매일,스타일,기획사,유통사,재생 시간,n
0,박지윤,Journey,싱글,2018.07.29,연주 음악,박지윤크리에이티브,Sony Music,08:00,(2)
1,박지윤,Journey,싱글,2018.07.29,연주 음악,박지윤크리에이티브,Sony Music,08:00,(2)


In [179]:
d3 = pd.concat([d1,d2], axis = 1)
d3

,title_song_or_not,title,artist,bugs_score,artist,album_title,앨범 종류,발매일,스타일,기획사,유통사,재생 시간,n
0,1,Letter from,박지윤,선호도 5점,박지윤,Journey,싱글,2018.07.29,연주 음악,박지윤크리에이티브,Sony Music,08:00,(2)
1,1\n[타이틀곡],Journey,박지윤,선호도 5점,박지윤,Journey,싱글,2018.07.29,연주 음악,박지윤크리에이티브,Sony Music,08:00,(2)


In [163]:
d5 = d3.copy()

In [107]:
d5.sort_index(axis=1)

,album_title,artist,artist,bugs_score,n,title,title_song_or_not,고음질,기획사,발매일,스타일,앨범 종류,유통사,장르,재생 시간
0,귀띔,위아영 (WeAreYoung),위아영 (WeAreYoung),선호도 5점,(2),귀띔,1\n[타이틀곡],"FLAC 16bit, 24bit24bit FLAC 음원 스펙트로그램 보기 하이 레졸루션",엠와이뮤직,2018.07.29,"포크 팝, 인디 팝",싱글,㈜ 카카오 M,"인디, 발라드/댄스/팝",06:28
1,귀띔,위아영 (WeAreYoung),위아영 (WeAreYoung),선호도 1점,(2),귀띔 (Inst.),2,"FLAC 16bit, 24bit24bit FLAC 음원 스펙트로그램 보기 하이 레졸루션",엠와이뮤직,2018.07.29,"포크 팝, 인디 팝",싱글,㈜ 카카오 M,"인디, 발라드/댄스/팝",06:28


In [184]:
df_june

,id,quantity,a2,a3
0,4,4675,0,34
1,5,56,45,78
2,6,2,2,34


In [194]:
df_may = pd.DataFrame({'quantity' : [6,6,7,2], 'a1' : [79,563,152,12],'id' : [1,2,3,7]})
df_may

,quantity,a1,id
0,6,79,1
1,6,563,2
2,7,152,3
3,2,12,7


In [195]:
pd.concat([df_june, df_may], sort = True).reset_index(drop = True)

,a1,a2,a3,id,quantity
0,NaN,0.0,34.0,4,4675
1,NaN,45.0,78.0,5,56
2,NaN,2.0,34.0,6,2
3,79.0,NaN,NaN,1,6
4,563.0,NaN,NaN,2,6
5,152.0,NaN,NaN,3,7
6,12.0,NaN,NaN,7,2


In [199]:
d3

,title_song_or_not,title,artist,bugs_score,artist,album_title,앨범 종류,발매일,스타일,기획사,유통사,재생 시간,n
0,1,Letter from,박지윤,선호도 5점,박지윤,Journey,싱글,2018.07.29,연주 음악,박지윤크리에이티브,Sony Music,08:00,(2)
1,1\n[타이틀곡],Journey,박지윤,선호도 5점,박지윤,Journey,싱글,2018.07.29,연주 음악,박지윤크리에이티브,Sony Music,08:00,(2)


In [229]:
d4 = pd.DataFrame(columns = ['title_song_or_not', 'title', 'artist', 'bugs_score', 'artist',
       'album_title', '앨범 종류', '발매일', '장르', '스타일', '기획사', '유통사', '재생 시간',
       '고음질', 'n'])
d4.

,title_song_or_not,title,artist,bugs_score,artist,album_title,앨범 종류,발매일,장르,스타일,기획사,유통사,재생 시간,고음질,n


In [230]:
d3

,title_song_or_not,title,artist,bugs_score,artist,album_title,앨범 종류,발매일,스타일,기획사,유통사,재생 시간,n,장르,고음질
0,1,Letter from,박지윤,선호도 5점,박지윤,Journey,싱글,2018.07.29,연주 음악,박지윤크리에이티브,Sony Music,08:00,(2),NaN,NaN
1,1\n[타이틀곡],Journey,박지윤,선호도 5점,박지윤,Journey,싱글,2018.07.29,연주 음악,박지윤크리에이티브,Sony Music,08:00,(2),NaN,NaN


In [232]:
d3.shape

(2, 15)

In [233]:
d4.shape

(0, 15)

In [231]:
pd.concat([d4, d3], sort = True).reset_index(drop = True)

AssertionError: Number of manager items must equal union of block items
# manager items: 14, # tot_items: 15

# 한 페이지짜리 함수

In [227]:
def crawling_one_page(pageNo) :
    pageNo = str(pageNo)
    d4 = pd.DataFrame(columns = ['title_song_or_not', 'title', 'artist', 'bugs_score', 'artist',
       'album_title', '앨범 종류', '발매일', '장르', '스타일', '기획사', '유통사', '재생 시간',
       '고음질', 'n'])#final_dataframe
    
    for idx in range(70) :
        
        dr.get('https://music.bugs.co.kr/newest/album/kpoppicked?page='+pageNo)
    
        container = dr.find_element_by_id('container')
        section = container.find_element_by_css_selector('section[class = "sectionPadding"]')
        div  = section.find_element_by_css_selector('div')
        li = div.find_elements_by_css_selector('li')[idx]
    
        music_info = {}
        
        
        fig = li.find_element_by_css_selector('figcaption')
        
        #artist
        p = fig.find_element_by_css_selector('p[class = "artist"]')

        try : #링크로 걸려있는 artist
            music_info['artist'] = [p.find_element_by_css_selector('a').text]

        except :
            music_info['artist'] = [p.find_element_by_css_selector('span').text]
            
        #album title
        a = fig.find_element_by_css_selector('a')
        music_info['album_title'] = [a.get_attribute('title')]
        
        #click 앨범 정보 페이지로 이동
        a.click()
        
        div = dr.find_element_by_class_name('basicInfo')
        tbody = div.find_element_by_css_selector('tbody')
        trs = tbody.find_elements_by_css_selector('tr')
        

        for tr in trs[1:] : #except artist
            th = tr.find_element_by_css_selector('th')
            td = tr.find_element_by_css_selector('td')

            music_info[th.text] = [td.text]

        
        #수록곡
        section = dr.find_element_by_css_selector('section[class = "sectionPadding contents track"]')
        music_info['n'] = [section.find_element_by_css_selector('p').text] #수록곡 개수
        
        table = section.find_element_by_css_selector('table')
        tbd = table.find_element_by_css_selector('tbody')
        trs = tbd.find_elements_by_css_selector('tr')
        
        music_list = []
        for tr in trs :
            ps = tr.find_elements_by_css_selector('p')

            aa = []
            #서비스를 지원하지 않는 곡들 처리
            try :
                tr.find_element_by_css_selector('input[disabled = "disabled"]')
            except :

                for p in ps :

                    aa.append(p.text)

                music_list.append(aa)
                
        d1 = pd.DataFrame(music_list,columns = ['title_song_or_not','title','artist','bugs_score'])
        d2 = pd.DataFrame(music_info)
        d2 = pd.DataFrame(np.repeat(np.array(d2),len(d1), axis = 0), columns = music_info.keys())
        d3 = pd.concat([d1,d2], axis = 1)
        
        d4 = pd.concat([d4,d3], sort = False).reset_index(drop = True)
        
        if((idx+1) % 5 == 0) : #다섯번째마다 저장
            print(idx+1, 'album...')
            d4.to_csv('pageNo_'+pageNo+'_'+(idx+1)+'.csv')
        

In [228]:
crawling_one_page(100)

AssertionError: Number of manager items must equal union of block items
# manager items: 14, # tot_items: 15

# checking

In [38]:
import os
os.listdir('./')

['.ipynb_checkpoints', '1.Data_Crawling_mini.ipynb', 'pageNo_100.csv']

In [40]:
df = pd.read_csv('./pageNo_100.csv')
df

,Unnamed: 0,title_song_or_not,title,artist,bugs_score,artist.1,album_title,앨범 종류,발매일,장르,스타일,기획사,유통사,재생 시간,고음질,n
0,0,1\r\n[타이틀곡],"우린 나란히 앉아 (With 정소안, 변흥수)",새봄(Saevom),선호도 5점,새봄(Saevom),우린 나란히 앉아,싱글,2018.07.29,"인디, 발라드/댄스/팝",인디 팝,어느새봄,포크라노스,03:44,FLAC 16bit,(1)
